<a href="https://colab.research.google.com/github/navroz-lamba/DS-Unit-2-Linear-Models/blob/master/Assignment_214_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# importing the dataset 
df = pd.read_csv('https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/burritos/burritos.csv', 
                 parse_dates=['Date'],
                index_col='Date')
df.head(25)

,Location,Burrito,Neighborhood,Address,URL,Yelp,Google,Chips,Cost,Hunger,...,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-18,Donato's taco shop,California,Miramar,6780 Miramar Rd,http://donatostacoshop.net/,3.5,4.2,NaN,6.49,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-24,Oscar's Mexican food,California,San Marcos,225 S Rancho Santa Fe Rd,http://www.yelp.com/biz/oscars-mexican-food-sa...,3.5,3.3,NaN,5.45,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-24,Oscar's Mexican food,Carnitas,NaN,NaN,NaN,NaN,NaN,NaN,4.85,1.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-24,Oscar's Mexican food,Carne asada,NaN,NaN,NaN,NaN,NaN,NaN,5.25,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-27,Pollos Maria,California,Carlsbad,3055 Harding St,http://pollosmaria.com/,4.0,3.8,x,6.59,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-28,Pollos Maria,combo chicken,NaN,NaN,NaN,NaN,NaN,NaN,6.99,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-30,Nico's Taco Shop,California,Carmel Valley,3860 Valley Centre Dr #404,http://www.yelp.com/biz/nicos-taco-shop-san-diego,3.0,2.9,NaN,7.19,1.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-30,Nico's Taco Shop,Carnitas,NaN,NaN,NaN,NaN,NaN,NaN,6.99,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-02-01,Los Primos Mexican Food,Monster California,UTC,7770 Regents Rd,http://www.primosmex.com/,3.0,3.7,x,9.25,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# df.Date.dt.date.min()

In [ ]:
# df.Date.dt.date.max()

In [ ]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [ ]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

In [ ]:
df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

df.head()

,Location,Burrito,Neighborhood,Address,URL,Yelp,Google,Chips,Cost,Hunger,...,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-18,Donato's taco shop,California,Miramar,6780 Miramar Rd,http://donatostacoshop.net/,3.5,4.2,NaN,6.49,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2016-01-24,Oscar's Mexican food,California,San Marcos,225 S Rancho Santa Fe Rd,http://www.yelp.com/biz/oscars-mexican-food-sa...,3.5,3.3,NaN,5.45,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2016-01-24,Oscar's Mexican food,Carnitas,NaN,NaN,NaN,NaN,NaN,NaN,4.85,1.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2016-01-24,Oscar's Mexican food,Asada,NaN,NaN,NaN,NaN,NaN,NaN,5.25,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2016-01-27,Pollos Maria,California,Carlsbad,3055 Harding St,http://pollosmaria.com/,4.0,3.8,x,6.59,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [ ]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [ ]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [ ]:
# split into train and test 
filt = df.index < '2018-01-01'
df_train = df.loc[filt]
df_test = df.loc[~filt]


In [ ]:
df_train.isna().sum().sort_values(ascending = False)

Queso             383
Zucchini          382
Carrots           382
Ham               382
Lobster           382
Sushi             381
Corn              380
Bacon             380
Mushroom          380
Chile relleno     379
Taquito           379
Nopales           379
Egg               379
Fish              377
Salsa.1           377
Pineapple         376
NonSD             376
Tomato            376
Bell peper        376
Cabbage           375
Lettuce           372
Avocado           370
Cilantro          368
Onion             366
Chicken           363
Shrimp            362
Mass (g)          361
Density (g/mL)    361
Chips             358
Unreliable        353
Beans             351
Rice              348
Sauce             345
Pork              336
Yelp              299
Google            299
Sour cream        294
Fries             259
Guac              237
Pico              232
Cheese            229
Beef              208
Circum            135
Volume            135
Length            134
Salsa     

In [ ]:
# since all the NaN values under categorical dtypes means they were not added to the burrito, 
#i am replacing them with not added so we could still use those to predict our model
cat_attribs = df_train.select_dtypes(include='object')

for value in cat_attribs:
    df_train[value].fillna('''Wasn't Added''', inplace= True)

/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [ ]:
df_train.dtypes

Burrito            object
Yelp              float64
Google            float64
Chips              object
Cost              float64
Hunger            float64
Mass (g)          float64
Density (g/mL)    float64
Length            float64
Circum            float64
Volume            float64
Tortilla          float64
Temp              float64
Meat              float64
Fillings          float64
Meat:filling      float64
Uniformity        float64
Salsa             float64
Synergy           float64
Wrap              float64
Unreliable         object
NonSD              object
Beef               object
Pico               object
Guac               object
Cheese             object
Fries              object
Sour cream         object
Pork               object
Chicken            object
Shrimp             object
Fish               object
Rice               object
Beans              object
Lettuce            object
Tomato             object
Bell peper         object
Carrots            object
Cabbage     

In [ ]:
# looking at the NaN values again
df_train.isna().sum().sort_values(ascending = False)

Queso             383
Density (g/mL)    361
Mass (g)          361
Yelp              299
Google            299
Circum            135
Volume            135
Length            134
Salsa              22
Temp               20
Meat               12
Meat:filling        8
Cost                7
Hunger              3
Fillings            2
Uniformity          2
Synergy             2
Wrap                2
NonSD               0
Guac                0
Pico                0
Cheese              0
Beef                0
Great               0
Unreliable          0
Sour cream          0
Tortilla            0
Chips               0
Fries               0
Pork                0
Zucchini            0
Chicken             0
Corn                0
Avocado             0
Sushi               0
Bacon               0
Mushroom            0
Egg                 0
Lobster             0
Nopales             0
Chile relleno       0
Ham                 0
Pineapple           0
Taquito             0
Onion               0
Cilantro  

In [ ]:
# we will drop Queso as all the values are NaN
# We will also delete the Density, Mass, yelp and Google as 90% or more of the values are NaN
df_train = df_train.drop(['Queso', 'Density (g/mL)', 'Mass (g)', 'Yelp', 'Google','Chips'], axis=1)
df_train.head()

,Burrito,Cost,Hunger,Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,...,Nopales,Lobster,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-18,California,6.49,3.0,NaN,NaN,NaN,3.0,5.0,3.0,3.5,...,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,False
2016-01-24,California,5.45,3.5,NaN,NaN,NaN,2.0,3.5,2.5,2.5,...,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,False
2016-01-24,Carnitas,4.85,1.5,NaN,NaN,NaN,3.0,2.0,2.5,3.0,...,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,False
2016-01-24,Asada,5.25,2.0,NaN,NaN,NaN,3.0,2.0,3.5,3.0,...,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,False
2016-01-27,California,6.59,4.0,NaN,NaN,NaN,4.0,5.0,4.0,3.5,...,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,Wasn't Added,True


we will take care of the rest of the numerical NaN values with the simple imputer  

# split into train, test and val

In [ ]:
X = df_train.drop('Great', axis=1)
y = df_train['Great']
df_train.shape

(383, 52)

In [ ]:
# Train set 
cutoff_train = '2017-01-01'
filt= (X.index < cutoff_train)

X_train, y_train = X.loc[filt], y.loc[filt]

# val set 
X_val, y_val = X.loc[~filt], y.loc[~filt]

In [ ]:
X_train.shape[0] + X_val.shape[0] == df_train.shape[0]

True

# lets build a pipeline 
### separating categorical and numerical attributes 

In [ ]:
df_cat = X_train.select_dtypes(include='object')
df_cat.columns

Index(['Burrito', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac', 'Cheese',
       'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish', 'Rice',
       'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots', 'Cabbage',
       'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito', 'Pineapple', 'Ham',
       'Chile relleno', 'Nopales', 'Lobster', 'Egg', 'Mushroom', 'Bacon',
       'Sushi', 'Avocado', 'Corn', 'Zucchini'],
      dtype='object')

In [ ]:
df_cat_val = X_val.select_dtypes(include='object')
df_cat_val.columns

Index(['Burrito', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac', 'Cheese',
       'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish', 'Rice',
       'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots', 'Cabbage',
       'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito', 'Pineapple', 'Ham',
       'Chile relleno', 'Nopales', 'Lobster', 'Egg', 'Mushroom', 'Bacon',
       'Sushi', 'Avocado', 'Corn', 'Zucchini'],
      dtype='object')

In [ ]:
df_num = X_train.select_dtypes(exclude='object')
df_num.columns

Index(['Cost', 'Hunger', 'Length', 'Circum', 'Volume', 'Tortilla', 'Temp',
       'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa', 'Synergy',
       'Wrap'],
      dtype='object')

In [ ]:
pipeline_num = Pipeline([('imputer', SimpleImputer()),
                        ('scaler', StandardScaler())])

In [ ]:
pipeline_cat = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                         ("ohe", OneHotEncoder(sparse=False))])

In [ ]:
# now putting the pipelines together using ColumnTransformer
#generate a list of col and num attributes that we could call 

num_attribs = list(df_num)
cat_attribs = list(df_cat)

full_pipeline = ColumnTransformer([
                                  ('categorical_pipeline', pipeline_cat, cat_attribs),
                                    ('numerical_pipeline', pipeline_num, num_attribs)])

# transform the X_train 
df_prepared_train = full_pipeline.fit_transform(X_train)
df_prepared_val = full_pipeline.transform(X_val)

## Making one final pipeline with the logistic regression model 

In [ ]:
# we could add regularization to the model by adding the hyperparameter, C
log_reg = Pipeline([('full_pipeline', full_pipeline),
                  ('logistic Regression', LogisticRegression())])
# fitting the train test
log_reg.fit(X_train, y_train)

# log_reg.fit(X_val, y_val)

/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('full_pipeline',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('categorical_pipeline',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='most_frequent',
                                    

In [ ]:
print('Training Accuracy:', log_reg.score(X_train, y_train))
print('Validation Accuracy:', log_reg.score(X_val, y_val))

Training Accuracy: 0.9429530201342282
Validation Accuracy: 0.7647058823529411


## We see that our model is clearly overfitting and we would need regularization 

In [ ]:
# lets find the best value for C using cross validation 
log_reg = LogisticRegression(random_state=42)

parameters = {'C':[.001, .005, 1e-5, .01, .05, 1e-2, 1, 5,10]}

log_reg_cv = RandomizedSearchCV(log_reg, parameters, n_iter=9,cv=10, random_state=42 )
log_reg_cv.fit(df_prepared, y_train)

print(log_reg_cv.best_params_)
print(log_reg_cv.best_estimator_)

/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: Futur

/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: Futur

{'C': 0.01}
LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)


/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
# using the best_estimator_ lets update the pipeline
log_reg = Pipeline([('full_pipeline', full_pipeline),
                  ('logistic Regression', LogisticRegression(**log_reg_cv.best_params_))])
# fitting the train test
log_reg.fit(X_train, y_train);

/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# Score with the regularized Logistic Regression model 

In [ ]:
print('Training Accuracy:', log_reg.score(X_train, y_train))
print('Validation Accuracy:', log_reg.score(X_val, y_val))

Training Accuracy: 0.8959731543624161
Validation Accuracy: 0.9058823529411765


In [ ]:
!pip install feature-engine

     |████████████████████████████████| 8.4MB 2.1MB/s eta 0:00:01
     |████████████████████████████████| 7.2MB 2.9MB/s eta 0:00:01
  Found existing installation: statsmodels 0.10.1
    Uninstalling statsmodels-0.10.1:
      Successfully uninstalled statsmodels-0.10.1
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3
